<a href="https://colab.research.google.com/github/offcool/DataEnginner_DataCampCert/blob/main/Copy_of_DEPracticalExamDatacamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

def merge_all_data(user_health_file, supplement_usage_file, experiments_file, user_profiles_file):
    user_profiles_df = pd.read_csv('user_profiles.csv')
    user_health_data_df = pd.read_csv('user_health_data.csv')
    supplement_usage_df = pd.read_csv('supplement_usage.csv')
    experiments_df = pd.read_csv('experiments.csv')

    bins = [0, 17, 25, 35, 45, 55, 65, np.inf]
    labels = ['Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']
    user_profiles_df['user_age_group'] = pd.cut(user_profiles_df['age'], bins=bins, labels=labels, right=False)
    user_profiles_df['user_age_group'] = user_profiles_df['user_age_group'].astype('category')
    user_profiles_df['user_age_group'] = user_profiles_df['user_age_group'].cat.add_categories('Unknown')
    user_profiles_df['user_age_group'].fillna("Unknown", inplace=True)
    user_profiles_df.drop(columns=['age'], inplace=True)

    user_health_data_df['activity_level'] = ((user_health_data_df['activity_level'] - 1) / 3) * 100
    user_health_data_df['sleep_hours'] = user_health_data_df['sleep_hours'].str.replace('[hH]', '').astype(float)

    supplement_usage_df['is_placebo'].astype(str)
    supplement_usage_df['dosage_grams'] = supplement_usage_df['dosage'] / 1000
    supplement_usage_df.drop(columns=['dosage', 'dosage_unit'], inplace=True)

    merged_df = pd.merge(user_profiles_df, user_health_data_df, on='user_id', how='left')
    # merged_df = pd.merge(merged_df, supplement_usage_df, left_on=['user_id','date'], right_on=['user_id','date'], how='left')
    merged_df = pd.merge(merged_df, supplement_usage_df, on=['user_id', 'date'], how='left')
    merged_df = pd.merge(merged_df, experiments_df, on='experiment_id', how='left')

    merged_df['supplement_name'].fillna('No intake', inplace=True)

    merged_df.drop(columns=['experiment_id', 'description'], inplace=True)

    merged_df.rename(columns={'name': 'experiment_name'}, inplace=True)

    merged_df['date'] = pd.to_datetime(merged_df['date'], format='%Y-%m-%d').dt.date

    new_order = ['user_id', 'date', 'email', 'user_age_group', 'experiment_name', 'supplement_name', 'dosage_grams',
                 'is_placebo', 'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level']

    merged_df[new_order]


    print(merged_df.head())
    print(merged_df.info())

    return merged_df

merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')

In [2]:
import pandas as pd
import numpy as np

def merge_all_data(user_health_data_path, supplement_usage_path, experiments_path, user_profiles_path):
    """
    Cleans and merges datasets into a single DataFrame.
    Parameters:
    - user_health_data_path (str): Path to 'user_health_data.csv'
    - supplement_usage_path (str): Path to 'supplement_usage.csv'
    - experiments_path (str): Path to 'experiments.csv'
    - user_profiles_path (str): Path to 'user_profiles.csv'

    Returns:
    - pd.DataFrame: Merged and cleaned DataFrame
    """
    # Load CSV files
    user_health = pd.read_csv(user_health_data_path)
    supplement_usage = pd.read_csv(supplement_usage_path)
    experiments = pd.read_csv(experiments_path)
    user_profiles = pd.read_csv(user_profiles_path)

    # Clean categorical and text data
    user_profiles['email'] = user_profiles['email'].str.strip().str.lower()
    experiments['name'] = experiments['name'].str.strip().str.title()
    supplement_usage['supplement_name'] = supplement_usage['supplement_name'].str.strip().str.title()

    # Clean and convert user health data
    user_health['sleep_hours'] = user_health['sleep_hours'].str.replace('[hH]', '', regex=True).astype(float)
    user_health['activity_level'] = ((user_health['activity_level'] - 1) / 3) * 100  # Normalizing activity level to percentage

    # Convert user profile columns
    user_profiles['age'] = pd.to_numeric(user_profiles['age'], errors='coerce')

    # Handle user age groups
    bins = [0, 17, 25, 35, 45, 55, 65, np.inf]
    labels = ['Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']
    user_profiles['user_age_group'] = pd.cut(user_profiles['age'], bins=bins, labels=labels, right=True)
    user_profiles['user_age_group'] = user_profiles['user_age_group'].cat.add_categories(['Unknown']).fillna('Unknown')

    user_profiles_clean = user_profiles[['user_id', 'email', 'user_age_group']]

    # Process supplement usage
    supplement_usage['dosage_grams'] = supplement_usage['dosage'].apply(
        lambda x: float(x) / 1000 if isinstance(x, str) and 'mg' in x.lower() else float(x)  # Convert dosage to grams if in mg
    )
    supplement_usage['supplement_name'] = supplement_usage['supplement_name'].fillna('No intake')
    supplement_usage['experiment_id'] = supplement_usage['experiment_id'].replace('-', np.nan)

    # Merge supplement usage with experiments data
    supplement_usage = supplement_usage.merge(
        experiments[['experiment_id', 'name']],
        on='experiment_id',
        how='left'
    )
    supplement_usage.rename(columns={'name': 'experiment_name'}, inplace=True)
    supplement_usage['experiment_name'] = supplement_usage['experiment_name'].fillna(np.nan)
    supplement_usage['is_placebo'] = supplement_usage['is_placebo'].map({'true': True, 'false': False}).fillna(np.nan)

    # Convert 'date' columns to datetime format
    user_health['date'] = pd.to_datetime(user_health['date'], errors='coerce')
    supplement_usage['date'] = pd.to_datetime(supplement_usage['date'], errors='coerce')

    # Merge user health data with supplement usage
    merged_data = pd.merge(user_health, supplement_usage, on=['user_id', 'date'], how='left')
    merged_data['supplement_name'] = merged_data['supplement_name'].fillna('No intake')
    merged_data['dosage_grams'] = merged_data['dosage_grams'].fillna(np.nan)
    merged_data['is_placebo'] = merged_data['is_placebo'].fillna(np.nan)
    merged_data['experiment_name'] = merged_data['experiment_name'].fillna(np.nan)

    # Merge with user profiles
    final_data = pd.merge(merged_data, user_profiles_clean, on='user_id', how='left')

    # Select and reorder columns
    final_columns = [
        'user_id', 'date', 'email', 'user_age_group', 'experiment_name',
        'supplement_name', 'dosage_grams', 'is_placebo',
        'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level'
    ]
    final_data = final_data[final_columns]

    # Ensure no missing values in critical columns (user_id, date, email)
    final_data = final_data.dropna(subset=['user_id', 'date', 'email'])

    return final_data

# Example usage
#merged_df = merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')
#print(merged_df.head(100))
